In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
def base_preprocess(data, start_date, end_date, record_cnt = 2):
    
    #1. rename columns & revalue student number id 
    data.rename(columns = {'대출분관':'borrow_loc', 
                           '등록번호': 'book_id',
                           '학번 난수처리' : 'student_id',
                           '대출자신분': 'age',
                           '소장분관' : 'book_loc',
                           '소장서고' : 'book_loc_2',
                           '서명' : 'title',
                           '청구기호' : 'book_ddc',
                           '대출처리일자': 'borrow_date', 
                           '대출일자' : 'borrow_date2',
                           '반납예정일자' : 'excepted_return_date',
                           '반납일자' : 'return_date2',
                           '반납처리일자' :'return_date', 
                           '대출상태' : 'borrow_state',
                           '대출유형' : 'borrow_type' ,
                           '반납처리수단' : 'return_method', 
                           '반납유형' : 'return_type',
                           '서지번호' :'seoji_num' ,
                           '반납유형주기':'return_type_cycle', 
                           '연체일': 'late_day',
                           '예약수': 'reserve_amount',
                           '딸림주기': 'ddalim_cycle' ,
                           '대출처리수단': 'borrow_method' ,
                           '대출자 소속': 'major'
                          }, inplace=True)
    
    data['student_id'] = data['student_id']*10000000000
    data['student_id'] = data['student_id'].astype('int')
    data['student_id'] = data['student_id'].astype('str')
    
    
    #2. delete except useful columns
    data = data[['book_id', 'student_id','book_loc', 'book_loc_2', 'title', 
                 'book_ddc', 'borrow_date', 'return_date', 
                 'borrow_state','borrow_type','return_type','seoji_num','major']]
    
    
    #3. return_type : return type delete of lost
    data = data[data['return_type']!='분실신고반납']

    #4. borrow_state : leave only return complete  
    data = data[data['borrow_state']== '반납완료']
    
    #5. borrow_type : only leave correct borrowing 
    data = data[data['borrow_type'].isin(['일반대출', '예약도서대출', '무인대출반납']) == True]

    #6. major: delete weird majors 
    data = data[data['major'].isin(['중앙도서관', '학술정보지원팀', '외부이용자']) == False]

    #7. book: delete thesis, workbooks and dvd
    data = data[~data['book_loc_2'].isin(['학위논문(제2서고)', '4층 과제도서', '수험서 코너',
                                          '과제도서(Service+ Center)','과제도서', 
                                          '4층 취업자료코너','3층 미디어자료실', '미디어자료실', 
                                          '일반자료실취업서가(3층)','건축학 비도서'])]
    
    #8. books: which does not have ddc call numbers
    data = data[data['book_ddc'].isnull() != True]
    
    
    #9. student: with more than 2 different borrow records
    data_grp = data.groupby('student_id').count()
    data_grp = data_grp[['borrow_date']]
    data_grp.sort_values('borrow_date', inplace = True)
    over_record_cnt = data_grp[data_grp['borrow_date']>= record_cnt]
    
    filtered_data = data[data['student_id'].isin(over_record_cnt.index)]

    
    #10. filtering by borrow and return date
    filtered_data = filtered_data[(filtered_data['borrow_date'] >= start_date) & (filtered_data['return_date'] < end_date)]
    
    #11. remain columns for prediction
    filtered_data = filtered_data[['book_id', 'student_id', 'title', 'book_ddc', 'major']]
    
    filtered_data.reset_index(inplace = True, drop = True)
    return filtered_data

def make_student_table(data):
    student_major = data[['student_id', 'major']]
    student_major = student_major.drop_duplicates(['student_id'], keep='first')
    student_major.sort_values('student_id', inplace=True)
    student_major.reset_index(inplace=True, drop=True)
    
    ## generalizing level of major as a colleage unit
    school_dict = {'None' : ['역사교육과', '동양철학과','메카트로닉스공학협동과정'],
                   '공과대학' : ['공학계열','화학공학/고분자공학부','화학공학부','화학공학','고분자공학부', '고분자시스템공학과', 
                             '신소재공학부', '재료공학과' ,'기계공학부', '건설환경공학부', '시스템경영공학과','건축학과',
                             '나노공학과', '건축토목공학부', '건축공학과','사회환경시스템공학과' ,'조경학과'],
                   '문과대학': ['인문과학계열', '인문학부' , '국어국문학과' , '영어영문학과',  '프랑스어문학과','중어중문학과',
                            '독어독문학과', '러시아어문학과','한문학과','사학과','철학과','문헌정보학과'],
                   '사회과학대학' : ['행정학과','행정학전공', '정치외교학과','미디어커뮤니케이션학과','사회학과','사회복지학과',
                               '신문방송학과','심리학과', '소비자학과','소비자가족학과','아동·청소년학과','글로벌리더학부',
                               '법학과','사회과학계열'],
                   '경제대학' : ['경제학과' , '통계학과','글로벌경제학과'],
                   '경영대학' : ['경영학과','글로벌경영학과' ,'경영학부'],
                   '사범대학' : ['교육학과','한문교육과','수학교육과','컴퓨터교육과'],
                   '예술대학' : ['미술학과','디자인학과','무용학과','영상학과','연기예술학과','의상학과','미술학전공','영상학전공','디자인학전공'],
                   '자연과학대학' : ['생명과학과', '수학과', '물리학과', '화학과','자연과학계열','생명과학전공','수학전공','화학전공'],
                   '정보통신대학' : ['전자전기공학부', '반도체시스템공학과' ,'전자전기공학전공','정보통신공학부'],
                   '의과대학' : ['의학과', '의예과'],
                   '생명공학대학' : ['식품생명공학과' , '바이오메카트로닉스학과', '융합생명공학과','유전공학과','식품생명공학전공'],
                   '소프트웨어대학' : ['소프트웨어학과', '컴퓨터공학과','전자전기.컴퓨터공학계열','컴퓨터공학전공'],
                   '성균융합원' : ['글로벌바이오메디컬공학과','글로벌융합학부' ,'데이터사이언스융합전공','인공지능융합전공',
                              '컬처앤테크놀로지융합전공','글로벌바이오메디컬엔지니어링학과']}
    
    
    def find_college(major):
        college_list = sorted(list(school_dict.keys()))
        
        for key, value in school_dict.items():
            if major in value:
                return [key, college_list.index(key)]
            
            
    temp = student_major['major'].apply(lambda x: find_dept(x), axis = 1, result_type = 'expand')
    student = pd.concat([student_major, temp], axis = 1)
    student = student[student['college'].isnull() != True]
    student.reset_index(inplace = True, drop = True)
    
    return student


def make_book_table(data):
    book_df = data[['book_id', 'title', 'book_ddc']]
    book_df = book_df.drop_duplicates(['book_id'], keep='first')
    book_df.sort_values('book_id', inplace=True)
    book_df.reset_index(inplace = True, drop = True)
    
    return book_df


In [ ]:
if __name__ == '__main__':
    raw_data = pd.read_csv('../tot_1519.csv')
    data = base_preprocess(raw_data, '2019-01-01', '2020-01-01')
    student_df = make_student_table(data)
    
    data = data[data['student_id'].isin(student_df['student_id']) == True]
    book_df = make_book_table(data)
    
    ## save df
    student_df.to_csv('student.csv')
    book_df.to_csv('book.csv')
    data.to_csv('data.csv')